In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os
from tensorflow.keras import layers as L
import librosa
from librosa.display import specshow as ss
from sklearn.model_selection import train_test_split as tts

In [ ]:
capuchin= '../input/z-by-hp-unlocked-challenge-3-signal-processing/Parsed_Capuchinbird_Clips'
not_capuchin = '../input/z-by-hp-unlocked-challenge-3-signal-processing/Parsed_Not_Capuchinbird_Clips'

In [ ]:
def load_mel_spectrogram(filepath):
    file,sr = librosa.load(filepath, sr = 16000, mono=True)
    padders = np.zeros((80000)-np.shape(file)[0])
    file = np.concatenate((padders, file),axis = 0)
    mel = np.array(librosa.feature.melspectrogram(file, sr = 16000))
    mel = tf.expand_dims(mel,axis = -1)
    return mel

In [ ]:
capuchin_calls = '../input/z-by-hp-unlocked-challenge-3-signal-processing/Parsed_Capuchinbird_Clips'
not_capuchin_calls = '../input/z-by-hp-unlocked-challenge-3-signal-processing/Parsed_Not_Capuchinbird_Clips'
filepaths = []
labels = []
for i in os.listdir(capuchin_calls):
    filepaths.append(capuchin_calls+'/'+i)
    labels.append(1)
for i in os.listdir(not_capuchin_calls):
    filepaths.append(not_capuchin_calls+'/'+i)
    labels.append(0)

In [ ]:
for i in range(len(filepaths)):
    filepaths[i] = load_mel_spectrogram(filepaths[i])

In [ ]:
labels = np.array(labels).reshape(-1,1)

In [ ]:
filepaths = np.array(filepaths)

In [ ]:
filepaths.shape
labels.shape

In [ ]:
xtrain,xtest,ytrain,ytest = tts(filepaths,labels, random_state=32,train_size=0.85,test_size=0.15)

In [ ]:
print(xtrain.shape,ytrain.shape)

In [ ]:
def make_model(input_shape):
    input_ = L.Input((input_shape))
    x = L.Conv2D(16,(2,2))(input_)
    x = L.Conv2D(16,(2,2))(x)
    x = L.Conv2D(16,(2,2))(x)
    x = L.MaxPool2D((2,2))(x)
    x = L.Conv2D(16,(2,2))(x)
    x = L.Conv2D(16,(2,2))(x)
    x = L.MaxPool2D((2,2))(x)
    x = L.Flatten()(x)
    x = L.Dense(128,activation = 'relu')(x)
    x = L.Dense(128,activation = 'relu')(x)
    output = L.Dense(1,activation = 'sigmoid')(x)
    model = tf.keras.Model(input_,output)
    return model
model = make_model(xtrain[0].shape)
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy',
                                                                        'mae','Recall'])

In [ ]:
x = model.fit(xtrain,ytrain,epochs = 5, verbose = 1)

In [ ]:
def pred(sample):
    return model.predict(np.array([sample]))[0][0]

In [ ]:
def exact(num):
    if num>0.85:
        return 1
    else:
        return 0
    
def take_count(pred_list):
    counter = pred_list[0]
    for i in range(1,len(pred_list)):
        if pred_list[i] == 1:
            if pred_list[i-1] == 1:
                continue
            else:
                counter += 1
        else:
            continue
    return counter


def prediction_on_a_single_clip(filepath):
    test_file,sr = librosa.load(filepath, sr = 16000)
    start = 0
    pred_list = []
    while start<test_file.shape[0]:
        end = start+80000
        test_sample = test_file[start:end]
        padders = np.zeros((80000)-np.shape(test_sample)[0])
        test_sample = np.concatenate((padders, test_sample),axis = 0)
        test_sample = np.array(librosa.feature.melspectrogram(test_sample, sr = 16000))
        test_sample = tf.expand_dims(test_sample,axis = -1)
        pred_list.append(exact(pred(test_sample)))
        start = end
    return take_count(pred_list)


In [ ]:
pred_list = prediction_on_a_single_clip('../input/z-by-hp-unlocked-challenge-3-signal-processing/Forest Recordings/recording_00.mp3')

In [ ]:
print(pred_list)

In [ ]:
test_dir = '../input/z-by-hp-unlocked-challenge-3-signal-processing/Forest Recordings'
final_dict = {'clip_name':[],'call_count':[]}
for i in os.listdir(test_dir):
    path = test_dir+'/'+i
    capuchin_calls = prediction_on_a_single_clip(path)
    final_dict['clip_name'].append(i)
    final_dict['call_count'].append(capuchin_calls)

    

In [ ]:
final_dict = pd.DataFrame(final_dict)

In [ ]:
final_dict

In [ ]:
final_dict.to_csv('result.csv')

In [ ]:
x